<a href="https://colab.research.google.com/github/Zekru69/-Hands-on-Lab-Getting-Started-with-GitHub/blob/main/SQL_final_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython-sql prettytable

import prettytable


prettytable.DEFAULT = 'DEFAULT'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.0 MB/s eta 0:00:00


In [2]:
!pip install pandas
import pandas as pd
import sqlite3

# Establish a connection to the SQLite database
conn = sqlite3.connect('FinalDB.db')

print("Connection to FinalDB.db established successfully.")

Connection to FinalDB.db established successfully.


In [3]:
%load_ext sql

In [4]:
import pandas as pd
import sqlite3

# Load data into Pandas dataframes
census_data = pd.read_csv('/content/ChicagoCensusData.csv')
chicago_public_schools = pd.read_csv('/content/ChicagoPublicSchools.csv')
chicago_crime_data = pd.read_csv('/content/ChicagoCrimeData.csv')

# Establish a connection to the SQLite database
conn = sqlite3.connect('FinalDB.db')

# Load dataframes into the database as tables
census_data.to_sql('CENSUS_DATA', conn, if_exists='replace', index=False)
chicago_public_schools.to_sql('CHICAGO_PUBLIC_SCHOOLS', conn, if_exists='replace', index=False)
chicago_crime_data.to_sql('CHICAGO_CRIME_DATA', conn, if_exists='replace', index=False)

print("Data loaded into FinalDB.db successfully.")

Data loaded into FinalDB.db successfully.


Problem 1
Find the total number of crimes recorded in the CRIME table

In [7]:
import sqlite3

# Establish connection
conn = sqlite3.connect('FinalDB.db')

# Create a cursor object
cursor = conn.cursor()

# SQL query to count total crimes
query = "SELECT COUNT(*) FROM CHICAGO_CRIME_DATA;"

# Execute the query
cursor.execute(query)

# Fetch result
total_crimes = cursor.fetchone()[0]

print(f"Total number of crimes recorded: {total_crimes}")

# Close connection
conn.close()


Total number of crimes recorded: 533


Problem 2
List community area names and numbers with per capita income less than 11000.

In [8]:
import sqlite3
import pandas as pd

# Establish connection
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PER_CAPITA_INCOME
FROM CENSUS_DATA
WHERE PER_CAPITA_INCOME < 11000;
"""

# Execute the query and load results into a DataFrame
df = pd.read_sql(query, conn)

# Display the result
print(df)

# Close the connection
conn.close()


   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  PER_CAPITA_INCOME
0                   26.0  West Garfield Park              10934
1                   30.0      South Lawndale              10402
2                   37.0         Fuller Park              10432
3                   54.0           Riverdale               8201


Problem 3
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [9]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%MINOR%';
"""

# Execute the query and store results in a DataFrame
df = pd.read_sql(query, conn)

# Display results
print(df)

# Close connection
conn.close()


  CASE_NUMBER          PRIMARY_TYPE                    DESCRIPTION
0    HL266884  LIQUOR LAW VIOLATION  SELL/GIVE/DEL LIQUOR TO MINOR
1    HK238408  LIQUOR LAW VIOLATION   ILLEGAL CONSUMPTION BY MINOR


Problem 4
List all kidnapping crimes involving a child?

In [10]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%';
"""

# Execute query and store results in a DataFrame
df = pd.read_sql(query, conn)

# Display results
print(df)

# Close connection
conn.close()


  CASE_NUMBER PRIMARY_TYPE               DESCRIPTION
0    HN144152   KIDNAPPING  CHILD ABDUCTION/STRANGER


Problem 5
List the kind of crimes that were recorded at schools. (No repetitions)

In [11]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT DISTINCT PRIMARY_TYPE
FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';
"""

# Execute query and store results in a DataFrame
df = pd.read_sql(query, conn)

# Display results
print(df)

# Close connection
conn.close()


             PRIMARY_TYPE
0                 BATTERY
1         CRIMINAL DAMAGE
2               NARCOTICS
3                 ASSAULT
4       CRIMINAL TRESPASS
5  PUBLIC PEACE VIOLATION


Problem 6
List the type of schools along with the average safety score for each type.

In [12]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT "School_Type", AVG("Safety_Score") AS Average_Safety_Score
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "School_Type";
"""

# Execute query and store results in a DataFrame
df = pd.read_sql(query, conn)

# Display results
print(df)

# Close connection
conn.close()


  "School_Type"  Average_Safety_Score
0   School_Type             49.504873


Problem 7
List 5 community areas with highest % of households below poverty line

In [13]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;
"""

# Execute query and store results in a DataFrame
df = pd.read_sql(query, conn)

# Display results
print(df)

# Close connection
conn.close()


   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  PERCENT_HOUSEHOLDS_BELOW_POVERTY
0                   54.0           Riverdale                              56.5
1                   37.0         Fuller Park                              51.2
2                   68.0           Englewood                              46.6
3                   29.0      North Lawndale                              43.1
4                   27.0  East Garfield Park                              42.4


Problem 8
Which community area is most crime prone? Display the coumminty area number only.

In [14]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT COMMUNITY_AREA_NUMBER
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1;
"""

# Execute the query
cursor = conn.cursor()
cursor.execute(query)

# Fetch result
most_crime_prone_area = cursor.fetchone()[0]

print(f"The most crime-prone community area number is: {most_crime_prone_area}")

# Close connection
conn.close()


The most crime-prone community area number is: 25.0


Problem 9
Use a sub-query to find the name of the community area with highest hardship index.

In [16]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);
"""

# Execute query
cursor = conn.cursor()
cursor.execute(query)

# Fetch result
highest_hardship_area = cursor.fetchone()[0]

print(f"The community area with the highest hardship index is: {highest_hardship_area}")

# Close connection
conn.close()


The community area with the highest hardship index is: Riverdale


Problem 10
Use a sub-query to determine the Community Area Name with most number of crimes?

In [17]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('FinalDB.db')

# Define SQL query
query = """
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);
"""

# Execute query
cursor = conn.cursor()
cursor.execute(query)

# Fetch result
most_crime_prone_area = cursor.fetchone()[0]

print(f"The community area with the most crimes is: {most_crime_prone_area}")

# Close connection
conn.close()


The community area with the most crimes is: Austin
